# Lint - Binary Operator Calculator

The syntax and evaluation:
```
3 => 3
4 => 4
Mul(3,3) => 3*3 => 9
Mul(4,4) => 4*4 => 16
Add(9,16) => 25
Add(Mul(3,3),Mul(4,4)) => Add(9,16) => 25
```

This section proposes two implementations which are first using data structure (Python's `list`) and second is using closure,procedural representation (i.e.: message passing style). The functions are organized as follow:

```
# Primitive operation we want to snarf from Python
x+y # the add operator
int # the integer
x*y # the mul operator

# constructor
Add(x,y) # create Add expression, x and y can be expression
Mul(x,y) # create Mul expression, x and y can be expression
int(x) # create a number

# predicate
is_Mul(e) # test e if mul expression
is_Add(e) # test e if add expression
is_Num(e) # test e if a number

# selector
augend(Add(x,y)) -> x
addend(Add(x,y)) -> y
multiplier(Mul(x,y)) -> x
multiplicand(Mul(x,y)) -> y
```

## The `calc` interpreter

In [1]:
def calc(p):
    if is_Num(p):
        return p
    elif is_Add(p):
        return calc(augend(p)) + calc(addend(p))
    elif is_Mul(p):
        return calc(multiplier(p)) * calc(multiplicand(p))

def is_Num(e):
    'is_Num(int(x)) -> True'
    ...
def is_Add(e):
    'is_Add(Add(x,y)) -> True'
    ...
def augend(e):
    'augend(Add(x,y)) -> x'
    ...
def addend(e):
    'addend(Add(x,y)) -> y'
    ...
def is_Mul(e):
    'is_Mul(Mul(x,y)) -> True'
    ...
def multiplier(e):
    'multiplier(Mul(x,y)) -> x'
    ...
def multiplicand(e):
    'multiplicand(Mul(x,y)) -> y'
    ...

## Data Structure

In [2]:
def is_Num(e):
    return isinstance(e,int)

def Add(x,y):
    return ['+',x,y]
def is_Add(e):
    return isinstance(e,list) and e[0] == '+'
def augend(e):
    return e[1]
def addend(e):
    return e[2]

def Mul(x,y):
    return ['*',x,y]
def is_Mul(e):
    return isinstance(e,list) and e[0] == '*'
def multiplier(e):
    return e[1]
def multiplicand(e):
    return e[2]

def recur_repr(e):
    if is_Add(e):
        return f'Add({recur_repr(augend(e))},{recur_repr(addend(e))})'
    elif is_Mul(e):
        return f'Mul({recur_repr(multiplier(e))},{recur_repr(multiplicand(e))})'
    elif is_Num(e):
        return str(e)
    else:
        return ''

expression = Add(Mul(3,3),Mul(4,4))
print(recur_repr(expression),'=>',calc(expression))

Add(Mul(3,3),Mul(4,4)) => 25


## Message passing style

In [3]:
def is_Num(e):
    return isinstance(e,int)

def recur_repr(e):
    return e('repr') if not is_Num(e) else e

def Add(x,y):
    def dispatch(msg):
        if msg == 'augend':
            return x
        elif msg == 'addend':
            return y
        elif msg == 'annotation':
            return 'add'
        elif msg == 'repr':
            return f'Add({recur_repr(x)},{recur_repr(y)})'
    return dispatch
def is_Add(e):
    return e('annotation') == 'add'
def augend(e):
    return e('augend')
def addend(e):
    return e('addend')

def Mul(x,y):
    def dispatch(msg):
        if msg == 'multiplier':
            return x
        elif msg == 'multiplicand':
            return y
        elif msg == 'annotation':
            return 'mul'
        elif msg == 'repr':
            return f'Mul({recur_repr(x)},{recur_repr(y)})'
    return dispatch
def is_Mul(e):
    return e('annotation') == 'mul'
def multiplier(e):
    return e('multiplier')
def multiplicand(e):
    return e('multiplicand')

expression = Add(Mul(3,3),Mul(4,4))
print(recur_repr(expression),'=>',calc(expression))

Add(Mul(3,3),Mul(4,4)) => 25


# Object based Calculator

It would be very cumbersome and a lot of work to use methods in previous section as the language becomes larger. This is because we've to define constructor,selectors and predicates for each type of expression added to our language. The reptitive tasks of defining these is an opportunity to introduce convenient abstraction to sort thing out. 

In `python`, we may make use of `class` to define constructor,selectors,predicates and representation at once.

In [4]:
class Add:
    def __init__(self,x,y) -> None:
        self.left = x
        self.right = y
    def __repr__(self) -> str:
        return f'Add({self.left},{self.right})'

class Mul:
    def __init__(self,x,y) -> None:
        self.left = x
        self.right = y
    def __repr__(self) -> str:
        return f'Mul({self.left},{self.right})'

Num=int
def calc(e):
    if isinstance(e,Num):
        return e
    elif isinstance(e,Mul):
        return calc(e.left) * calc(e.right)
    elif isinstance(e,Add):
        return calc(e.left) + calc(e.right)
    
expression = Add(Mul(3,3),Mul(4,4))
print(repr(expression),'=>',calc(expression))

Add(Mul(3,3),Mul(4,4)) => 25


# Structural Pattern Matching
In writing interpter using `python`, it can be more convenient by using `match...case` statement as follows

In [5]:
Mul.__match_args__=('left','right')
Add.__match_args__=('left','right')
def calc(e):
    match e:
        case int(x):
            return x
        case Mul(left,right):
            return calc(left) * calc(right)
        case Add(left,right):
            return calc(left) + calc(right)
    
expression = Add(Mul(3,3),Mul(4,4))
print(repr(expression),'=>',calc(expression))

Add(Mul(3,3),Mul(4,4)) => 25


# `dataclass`
As a programmer, it is a virtue to be lazy; invest effort in reducing repetition tasks. Indeed, `python` provides a convenient way to define constructors,selectors,predicates,representation and match arguments at once; by using the decorator `dataclass`.

In later sections, we shall use the `dataclass` abstraction in developing interpreter.

In [6]:
from dataclasses import dataclass
@dataclass
class Exp: ...
Val = int|float
Expr = Exp|Val

@dataclass
class BinOp(Exp):
    left:Expr
    right:Expr

@dataclass
class Add(BinOp): ...

@dataclass
class Sub(BinOp): ...

@dataclass
class Mul(BinOp): ...

@dataclass
class Div(BinOp): ...

def calc(e:Expr):
    match e:
        case int(x)|float(x):
            return x
        case Mul(left,right):
            return calc(left) * calc(right)
        case Add(left,right):
            return calc(left) + calc(right)
        case Sub(left,right):
            return calc(left) - calc(right)
        case Div(left,right):
            return calc(left) / calc(right)

expression = Mul(
    Mul(Div(4,3),
        Mul(3,Mul(3,3))),
    Div(22,7))
print(repr(expression),'=>',calc(expression))

Mul(left=Mul(left=Div(left=4, right=3), right=Mul(left=3, right=Mul(left=3, right=3))), right=Div(left=22, right=7)) => 113.14285714285714


# Lvar

In previous section, we've design and implement the language `calc`. Say, the formula calculating the sphere volume can be expressed as follow:

$V(r) = \frac{4}{3}\pi r^3$

```
Mul(Mul(Div(4,3),Mul(3,Mul(3,3))),Div(22,7)) => 113.142
Mul(Mul(Div(4,3),Mul(5,Mul(5,5))),Div(22,7)) => 523.809
Mul(Mul(Div(4,3),Mul(7,Mul(7,7))),Div(22,7)) => 1437.333
```


To calculate the volume at radius $r=5$ using `calc`, then we may type the formula accordingly and replace the symbol accordingly. But we have to repeat this process whenever we use different value of radius. It would be better then the `calc` remember the formula and we only pass the value of radius to the formula to get the result. Furthermore, the program is obscured from its original meaning like `Div(22,7)` is the approximated value of $\pi$

However, the `calc` here is not that expressive as it does not support the notion of associate names with values; this is percisely the concept of variables. To support variables, our interpreter must somehow to store that information in somewhere. This section propose the implementation of environment that store association, and extend the language to support variables.

The syntax and evaluation are as follow:
```
Let(Var('x'),10,Add(Var('x'),10)),Env()
    => Add(Var('x'),10), Env([['x',10]])
        Var('x'), Env([['x',10]]) => 10
        10,Env() => 10
    => Add(10,10), Env([['x',10]])
    => 20
```

## Environment
The contract is as follow
```
create_env() => Env([])
extend_env(x,v,Env([*xs])) => Env([[x,v],*xs])
apply_env(x,Env([[y,v],*xs]))
    => v if x == y
    => apply_env(x,Env([*xs]))
apply_env(x,Env([])) => error
```

In [7]:
from dataclasses import dataclass
@dataclass
class Env:
    env:list[list[str,int|float]]
    def apply(self,var:str):
        match self:
            case Env([]):
                raise NameError()
            case Env([[x,v],*xs]) if x==var:
                return v
            case Env([_,*xs]):
                return Env([*xs]).apply(var)
    
    def extend(self,var:str,val:Val):
        return Env([(var,val),*self.env])

def create_env() -> Env:
    return Env([])

def apply_env(var:str,env:Env) -> Val:
    return env.apply(var)

def extend_env(var:str,v:Val,env:Env) -> Env:
    return env.extend(var,v)

env = extend_env('x',5,
                 extend_env('y',3,
                            extend_env('x',2,
                                       extend_env('z',7,create_env()))))
apply_env('z',env),apply_env('x',env),apply_env('y',env)

(7, 5, 3)

##  Add `let` to Lvar

In [8]:
@dataclass
class Var:
    id:str

@dataclass
class Let:
    var:Var
    val:Exp
    body:Exp

def interp(e,env):
    match e:
        case int(x)|float(x):
            return x
        case Var(id):
            return apply_env(id,env)
        case Mul(left,right):
            return interp(left,env) * interp(right,env)
        case Add(left,right):
            return interp(left,env) + interp(right,env)
        case Sub(left,right):
            return interp(left,env) - interp(right,env)
        case Div(left,right):
            return interp(left,env) / interp(right,env)
        case Let(str(id),val,body):
            return interp(body,extend_env(str(id),interp(val,env),env))

In [9]:
formula = Let('pi',Div(22,7),Mul(Div(4,3),Mul(Var('pi'),Mul(Var('r'),Mul(Var('r'),Var('r'))))))
expression = Let('r',3,formula)
print(interp(expression,create_env()))
expression = Let('r',5,formula)
print(interp(expression,create_env()))
expression = Let('r',7,formula)
print(interp(expression,create_env()))

113.14285714285714
523.8095238095237
1437.3333333333333


# Lproc

Though we've added variables to the language, it's not that expressive because we can't define the notion of formula in `Lvar`. Hence, we include the procedural abstraction to our language. The syntax and evaluation are as follow:

```py
Let('sqr', Proc('x',Mul(Var('x'),Var('x'))), Add(App(Var('sqr'),3), App(Var('sqr'),4))), Env([])
=> Add(App(Var('sqr'),3), App(Var('sqr'),4)), Env([['sqr',Proc('x',Mul(Var('x'),Var('x'))]])
    => App(Var('sqr'),3), Env([['sqr',Proc('x',Mul(Var('x'),Var('x'))]])
    => App(Proc('x',Mul(Var('x'),Var('x')), 3), Env([['sqr',Proc('x',Mul(Var('x'),Var('x'))]])
    => Mul(Var('x'),Var('x')), Env([['x',3],['sqr',Proc('x',Mul(Var('x'),Var('x'))]])
    => Mul(3,3) => 9
...
=> Add(9,16) => 25
```

In [10]:
@dataclass
class Proc(Exp):
    param:str
    body:Expr

@dataclass
class App(Exp):
    op:Expr
    arg:Expr

def interp(e,env):
    match e:
        case int(x)|float(x):
            return x
        case Mul(left,right):
            return interp(left,env) * interp(right,env)
        case Add(left,right):
            return interp(left,env) + interp(right,env)
        case Sub(left,right):
            return interp(left,env) - interp(right,env)
        case Div(left,right):
            return interp(left,env) / interp(right,env)
        case Let(str(id),val,body):
            return interp(body,extend_env(id,interp(val,env),env))
        case Proc(params,body):
            return Proc(params,body)
        case Var(id):
            return apply_env(id,env)
        case App(op,arg):
            match interp(op,env):
                case Proc(param,body):
                    return interp(body,extend_env(param,interp(arg,env),env))
        case _:
            raise NotImplementedError('interp, unexpected argument', e)

In [11]:
expression = Let('sqr', Proc('x', Mul(Var('x'),Var('x'))), 
                 Add(App(Var('sqr'),3), App(Var('sqr'),4)))
print(interp(expression,create_env()))
cube = Proc('x',Mul(Var('x'),Mul(Var('x'),Var('x'))))
volume = Proc('r',Let('pi',Div(22,7), Mul(Div(4,3),Mul(Var('pi'),App(Var('cube'),Var('r'))))))
# 4/3*22/7*(2**3)**3 = 2145.52...
expression = Let('cube',cube,Let('volume',volume, App(Var('volume'),App(Var('cube'),2))))
print(interp(expression,create_env()))

25
2145.523809523809


# Lif

The language `Lproc` is lack of control flow construct; it cannot selectively interpret the program. For example, it is nearly impossible to define the `abs` formula in `Lproc`

$$
abs(x) = \begin{cases}
      -x & \text{if $x < 0$}\\
      x & \text{otherwise}
    \end{cases}
$$
Therefore, to support such feature, the language shall include `if` construct and boolean expression (e.g.: `<,>,<=,>=,==`).

The added syntax and evaluation as follow:

```

```


In [12]:
@dataclass
class Lt(BinOp): ...
@dataclass
class Gt(BinOp): ...
@dataclass
class Le(BinOp): ...
@dataclass
class Ge(BinOp): ...
@dataclass
class Eq(BinOp): ...

Val = int|float|bool
Expr = Exp|Val

@dataclass
class If(Exp):
    pred:Expr
    conseq:Expr
    alter:Expr

def interp(e,env):
    match e:
        case int(x)|float(x)|bool(x):
            return x
        case Mul(left,right):
            return interp(left,env) * interp(right,env)
        case Add(left,right):
            return interp(left,env) + interp(right,env)
        case Sub(left,right):
            return interp(left,env) - interp(right,env)
        case Div(left,right):
            return interp(left,env) / interp(right,env)
        case Let(str(id),val,body):
            return interp(body,extend_env(id,interp(val,env),env))
        case Proc(params,body):
            return Proc(params,body)
        case Var(id):
            return apply_env(id,env)
        case App(op,arg):
            match interp(op,env):
                case Proc(param,body):
                    return interp(body,extend_env(param,interp(arg,env),env))
        case Lt(left,right):
            return interp(left,env) < interp(right,env)
        case Gt(left,right):
            return interp(left,env) > interp(right,env)
        case Le(left,right):
            return interp(left,env) <= interp(right,env)
        case Ge(left,right):
            return interp(left,env) >= interp(right,env)
        case Eq(left,right):
            return interp(left,env) == interp(right,env)
        case If(pred,conseq,alter):
            if interp(pred,env) is True:
                return interp(conseq,env)
            else:
                return interp(alter,env)
        case _:
            raise NotImplementedError('interp, unexpected argument', e)

In [13]:
abs = Proc('x',If(Lt(Var('x'),0),Sub(0,Var('x')),Var('x')))
expression = Let('abs',abs, App(Var('abs'),-10))
print(interp(expression,create_env()))
expression = Let('abs',abs, App(Var('abs'),10))
print(interp(expression,create_env()))
expression = Let('abs',abs, App(Var('abs'),0))
print(interp(expression,create_env()))
expression = Let('abs',abs, App(Var('abs'),-2))
print(interp(expression,create_env()))

10
10
0
2


# Marco Expander (a.k.a syntatic extension)

Observe that the semantic of `Let` can be dervied from `Proc` and `App` such that

```py
Let(x,v,body) = App(Proc(x,body),v)
```

There are a lot of constructs can be derived. For example, we can define `And` and `Or` operator using `If` such that

```py
And(p1,p2) = If(p1,p2,False)
Or(p1,p2) = If(p1,True,p2)
Not(p1) = If(p1,false,true)
```

The syntax and semantic of `cond` are as follow:

```py
Cond([[p1,e1],[p2,e2],...,e_n])
=> If(p1,e1,Cond([[p2,e2],...,e_n]))

Cond([[p,e],_e])
=> If(p,e,_e)

Cond([[p,e]])
=> If(p,e,false)
```

Instead of interpreting the `Cond` that way, we expand first the recursive case until it reaches the other base cases, we'll get

```py
Cond([[p1,e1],[p2,e2],...,[p_m,e_m],e_n])
=> If(p1,e1,If(p2,e2,If(...,If(p_m,e_m,e_n))))
```

Indeed, we can even interleave some naive optimizations during expanding such that
```py
If(true,conseq,alter) => conseq
If(false,conseq,alter) => alter
If(Not(e),conseq,alter) => If(e,conseq,alter)
```

In [14]:
@dataclass
class UnaryOp(Exp):
    e:Expr

@dataclass
class Not(UnaryOp):...
@dataclass
class And(BinOp):...
@dataclass
class Or(BinOp):...

@dataclass
class Cond(Exp):
    clauses:list[list[Expr,Expr]]

def expand(e):
    match e:
        case int()|float()|bool()|Var():
            return e
        case And(left,right):
            return expand(If(left,right,False))
        case Or(left,right):
            return expand(If(left,True,right))
        case If(Not(pred),conseq,alter):
            return expand(If(pred,alter,conseq))
        case If(True,conseq,_):
            return expand(conseq)
        case If(False,_,alter):
            return expand(alter)
        case If(pred,conseq,alter):
            return If(expand(pred),expand(conseq),expand(alter))
        case Cond([[pred,conseq],alter]):
            return expand(If(pred,conseq,alter))
        case Cond([[pred,conseq]]):
            return expand(If(pred,conseq,False))
        case Cond([[pred,conseq],*clauses]):
            return expand(If(pred,conseq,Cond([*clauses])))
        case Let(str(id),val,body):
            return expand(App(Proc(id,body),val))
        case App(op,arg):
            return App(expand(op),expand(arg))
        case Proc(param,body):
            return Proc(param,expand(body))
        case BinOp(left,right):
            return e.__class__(expand(left),expand(right))
        case _:
            raise NotImplementedError('expander, unexpected ',e)

In [15]:
abs = Proc('x',If(Lt(Var('x'),0),Sub(0,Var('x')),Var('x')))
abs = Proc('x',Cond([
    [Lt(Var('x'),0),Sub(0,Var('x'))],
    [Eq(Var('x'),0),Var('x')],
    Var('x'),
]))
exp = Let('abs',abs, App(Var('abs'),-10))
r = interp(expand(exp),create_env())
print(r)
exp = And(Not(False),Or(False,True))
r = expand(exp) # we can see that it's already evaluate the result before passing to interpreter
print(r)

10
True


# Recursion
The language requires no special mechanism in supporting recursion

In [16]:
exp = Let('sum',Proc('n',
                     If(Eq(Var('n'),0),
                        0,
                        Add(Var('n'),
                            App(Var('sum'),Sub(Var('n'),1))))),
          App(Var('sum'),10))
env = create_env()
interp(expand(exp),env)

55

# Conclusion

**Interpreter itself is just another program**

It's quite subtle that it's just another program that explain program's result. It's not necessary to rely on hardware to explain the meaning of program, but we can develop a program called interpreter to explain  the meaning of program.

> The intepreter, which determines the meaning of expressions in a programming language, is just another program. 
> 
> Hal Abelson, Structure and Interpretation of Computer Program

Indeed, the gba emulator is just another interpreter that execute the legacy code of gba games by interpreting the legacy machine language; yes, it's not our device directly execute these legacy bur rather, it's the emulator that interpreting these legacy code.

The study of this chatper belong to the field of programming languages, specifically the implementation and design.

**Abstraction**

We see how proper abstraction can ease the development and make the code more concise via pattern-matching and object-oriented techniques.

**Macro**

The macro expander allows smaller implementation of programming language. This is achieve through two functions `expand` and `interp` where `interp` implements the core constructs whereas `expand` implements the derived constructs. This is how marco in `Scheme` language works. Though, the notebook shows the application of macro via `python` but the ideas are the same.

Indeed, marco is an abstracton over programming language itself since it can extend the languages. It's a meta-language feature. For example, the pattern-matching in `Scheme` can be shipped by using macro, rather than being as a part of compiler.

**Interpreter as a way to study the semantics of programming languages**

**By implementing a programming langauge (interpreter), it helps us better understand why language is designed this way and the implications of features.**

In [17]:
exp = Let('a',3,
          Let('p',Proc('z',Var('a')),
              Let('f',Proc('x',App(Var('p'),0)),
              Let('a',5,
                  App(Var('f'),2)))))
interp(expand(exp),create_env())

5

Adapted example from EOPL
```
let a = 3
in let p = proc (z) a
    in let f = proc (x) (p 0)
        in let a = 5
            in (f 2)
```
or equivalent in `Lproc`
```py
Let('a',3,
    Let('p',Proc('z',Var('a')),
        Let('f',Proc('x',App(Var('p'),0)),
        Let('a',5,
            App(Var('f'),2)))))
```
or equivalent in `haskell` assuming you paste it in repl mode (`ghci`)
```haskell
:{
let a = 3
    in let p z = a
        in let f x = (p 0)
            in let a = 5
                in (f 2)
:}
```
or equivalent in `racket`
```
(let ([a 3])
    (let ([p (lambda (z) a)])
        (let ([f (lambda (x) (p 0))])
            (let ([a 5])
                (f 2)))))
```

What do we expect the result returns? According to the `Lproc`, it returns 5 but `haskell` and `racket` returns 3. This is because according to the implementation, `Lproc` extends the environment upon application. This causes that the interpreter look for the value of variable that's most recently bound to environment. This is known as *dynamic scoping*. Whereas, `haskell`,`racket` and most of modern programming languages follows *lexical scoping* or some variant of it.

Why *dynmaic scoping* is implemented in this notebook? Because it's easy where the interpreter only return the value of procedure itselt and only extend environment upon applied. Furthermore, recursion under this implementation doesn't require special treatment.

To support *lexical scoping* together with *closure*, the procedure and recursion require special treatment which complicates the implementation.

But why most of modern programming languages adopt *lexical scoping*?
- It allows optimization as all references of variables can pre-determined (hence the name is lexical)
- It's easier to read the program, as the variables implies by the scoping
- Dynamic scoped program is very hard to debug than lexical scoped program

**What's missing from this chapter**

- Doesn't include implementation of parser.
- Doesn't use inheritance as mean of abstraction to extend the language